This playbook check duplicated releases

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import re

In [2]:
pd.options.display.max_rows = 500

In [3]:
import os
import sys

releasy_module = os.path.abspath(os.path.join('..','..','..','dev','releasy2'))
if releasy_module not in sys.path:
    sys.path.insert(0, releasy_module)
    
import releasy
from releasy.miner_git import GitVcs
from releasy.miner import TagReleaseMiner, PathCommitMiner, RangeCommitMiner, TimeCommitMiner, VersionReleaseMatcher, VersionReleaseSorter, TimeReleaseSorter, VersionWoPreReleaseMatcher


In [4]:
projects = pd.read_pickle('projects.zip')

In [6]:
i = 0
p = 0
for project in projects.itertuples():
    path = os.path.abspath(os.path.join('..','..','..','repos2',project.Index))
    
  #  try:
    vcs = GitVcs(path)
    release_matcher = VersionWoPreReleaseMatcher()
    version_release_sorter = VersionReleaseSorter()
    release_miner = TagReleaseMiner(vcs, release_matcher)

    release_set = release_miner.mine_releases()

    if len(release_set.prefixes) > 1:
        print(f"{i+1:<2}: {project.Index} - {project.lang}")

        for prefix in release_set.prefixes:
            freq = release_set.prefixes.count(prefix)
            print(f" - {freq:3} {prefix}")
        i += 1

        prev_release = None
        for release in release_set:
            main_prefix = release_set.prefixes.mode()
            if release.name.prefix != main_prefix or (prev_release and prev_release.name.prefix != main_prefix):
                if prev_release and release.name.version == prev_release.name.version: 
                    if release.head.id == prev_release.head.id:
                        warn = "clone"
                    else:
                        warn = "check"
                else:
                    warn = ""
                print(f"{warn:6} {prev_release.name if prev_release else '':10} < {release.name:10} {project.Index:35}")
                p += 1
            prev_release = release
        print("\n\n")
        
        #if len(release_set.prefixes) > 1:
        #for release in release_set:
        #    print(f" - {release.name}")
        
   # except Exception as e:
    #    print(f" - error: {e}")

print(f"Total: {i:6} Releases to check: {p:6}")

1 : vuejs/vue - javascript
 -  38 None
 - 120 v
       v2.6.12    < 0.11.0     vuejs/vue                          
       0.11.0     < 0.11.1     vuejs/vue                          
       0.11.1     < 0.11.10    vuejs/vue                          
       0.11.10    < 0.11.2     vuejs/vue                          
       0.11.2     < 0.11.3     vuejs/vue                          
       0.11.3     < 0.11.4     vuejs/vue                          
       0.11.4     < 0.11.5     vuejs/vue                          
       0.11.5     < 0.11.6     vuejs/vue                          
       0.11.6     < 0.11.7     vuejs/vue                          
       0.11.7     < 0.11.8     vuejs/vue                          
       0.11.8     < 0.11.9     vuejs/vue                          
       0.11.9     < 0.12.0     vuejs/vue                          
       0.12.0     < 0.12.1     vuejs/vue                          
       0.12.1     < 0.12.10    vuejs/vue                          
       0.12.10

In [7]:
releases = pd.read_pickle("releases.zip")

In [8]:
# Remover duplicatas

In [9]:
dup = releases.groupby(['project','head']).count()
dup = dup[dup['version'] > 1].reset_index()[['project','head', 'version']]
dup.head(5)

,project,head,version
0,dotnet/efcore,1e01d4f9c2e8727c9b475552025b2d493993045d,2
1,facebook/react,4337c1c00609ec8d7ae399c736e9d37bb159fac5,3


In [10]:
releases[releases['head'].isin(dup['head'])][['time','head','commits','base_releases']]

time  \
project        name                                 
facebook/react 0.14.10  2020-10-14 19:32:49+00:00   
               v15.7.0  2020-10-14 19:32:49+00:00   
               v16.14.0 2020-10-14 19:32:49+00:00   
dotnet/efcore  v2.1.22  2020-07-28 17:14:02+00:00   
               v2.1.23  2020-07-28 17:14:02+00:00   

                                                             head  commits  \
project        name                                                          
facebook/react 0.14.10   4337c1c00609ec8d7ae399c736e9d37bb159fac5      772   
               v15.7.0   4337c1c00609ec8d7ae399c736e9d37bb159fac5        0   
               v16.14.0  4337c1c00609ec8d7ae399c736e9d37bb159fac5        0   
dotnet/efcore  v2.1.22   1e01d4f9c2e8727c9b475552025b2d493993045d        1   
               v2.1.23   1e01d4f9c2e8727c9b475552025b2d493993045d        0   

                        base_releases  
project        name                    
facebook/react 0.14.10     [v16.13.0]  
               v15.7.0      [0.14.10]  
               v16.14.0     [0.14.10]  
dotnet/efcore  v2.1.22      [v2.1.18]  
               v2.1.23      [v2.1.22]

In [11]:
dup = releases.groupby(['project','semantic_version']).count()
dup = dup[dup['version'] > 1].reset_index()[['project','head', 'semantic_version', 'version']]
dup.head(5)

,project,head,semantic_version,version
0,WordPress/WordPress,4,1.5.1,4
1,apache/dubbo,2,2.7.4,2
2,aspnetboilerplate/aspnetboilerplate,3,0.10.0,3
3,aspnetboilerplate/aspnetboilerplate,2,0.10.1,2
4,aspnetboilerplate/aspnetboilerplate,3,0.10.3,3


In [130]:
r = releases.reset_index()
r[(r['project'] == 'aspnetboilerplate/aspnetboilerplate') & (r['semantic_version'] == '0.10.3')]

,project,name,version,semantic_version,prefix,suffix,lang,head,time,commits,...,range_commits,range_base_releases,range_tpos,range_fpos,range_fneg,time_commits,time_base_releases,time_tpos,time_fpos,time_fneg
9665,aspnetboilerplate/aspnetboilerplate,v0.10.3.0,0.10.3.0,0.10.3,v,None,c#,ef643ebdd57fedd11f139af0f2ad91aa7acd913b,2016-07-27 12:38:19+00:00,32,...,32,[v0.10.1.2],32,0,0,32,[v0.10.1.2],32,0,0
9666,aspnetboilerplate/aspnetboilerplate,v0.10.3.1,0.10.3.1,0.10.3,v,None,c#,cf20629f9c0114a0eef1bce3a10e04aab9307598,2016-08-04 07:48:33+00:00,10,...,10,[v0.10.3.0],10,0,0,12,[v0.10.3.0],10,2,0
9667,aspnetboilerplate/aspnetboilerplate,v0.10.3.2,0.10.3.2,0.10.3,v,None,c#,627901822fd701508054e7fc0b58fbf8af0b8517,2017-04-28 06:56:29+00:00,2,...,2,[v0.10.3.1],2,0,0,731,[v0.10.3.1],2,729,0


In [88]:
releases.loc['Wox-launcher/Wox']

,version,prefix,suffix,lang,head,time,commits,merges,base_releases,base_releases_qnt,range_commits,range_base_releases,range_tpos,range_fpos,range_fneg,time_commits,time_base_releases,time_tpos,time_fpos,time_fneg
name,,,,,,,,,,,,,,,,,,,,
v1.0.0.219,1.0.0.219,v,None,c#,48fdb5301784cd8ccbe3d32bb1c881a88c4029e5,2014-08-14 14:21:07+00:00,491,76,[],0,491,[],491,0,0,588,[],491,97,0
1.0.1,1.0.1,None,None,c#,26c882f100439d2d80a1aed549ba8b7fc9822796,2014-10-13 09:14:53+00:00,19,4,[v1.0.0.219],1,19,[v1.0.0.219],19,0,0,38,[v1.0.0.219],19,19,0
1.0.2,1.0.2,None,None,c#,27d2c4762280f0ff3b050483d70073797e6ac6c7,2014-11-27 04:38:13+00:00,10,2,[1.0.1],1,10,[1.0.1],10,0,0,56,[1.0.1],10,46,0
v1.1.0,1.1.0,v,None,c#,89f36efc179c9a8ea4656b044e300b5308431b61,2014-12-18 04:23:07+00:00,59,8,"[1.0.2, 1.0.1]",2,59,[1.0.2],59,0,0,82,[1.0.2],41,41,18
v1.1.1,1.1.1,v,None,c#,de0923f26fb264bd6789236fe983d3db6fa1f93c,2014-12-23 14:22:20+00:00,9,0,[v1.1.0],1,9,[v1.1.0],9,0,0,18,[v1.1.0],9,9,0
v1.2.825,1.2.825,v,None,c#,7f2445d58f672c2ca1a57a92dbcf97a13cf2bd20,2016-04-18 22:04:41+00:00,504,83,[v1.1.1],1,504,[v1.1.1],504,0,0,1014,[v1.1.1],504,510,0
v1.2.857,1.2.857,v,None,c#,10259dcecc00d75a47e84f9764806e9d1b62b177,2016-04-24 01:53:45+00:00,38,4,[v1.2.825],1,38,[v1.2.825],38,0,0,70,[v1.2.825],35,35,3
v1.2.868,1.2.868,v,None,c#,f37a7846624af226684afaaad2d1053f8ea81f5e,2016-04-25 02:00:55+00:00,10,0,[v1.2.857],1,10,[v1.2.857],10,0,0,17,[v1.2.857],10,7,0
v1.2.875,1.2.875,v,None,c#,8d9f2423863a91a4b9209b9c2b7eae4e87b09af7,2016-04-26 01:40:23+00:00,12,0,[v1.2.868],1,12,[v1.2.868],12,0,0,24,[v1.2.868],12,12,0


In [33]:
releases[releases["commits"] == 0]

version prefix suffix  \
project                             name                                  
microsoft/TypeScript                v1.5.4          1.5.4      v   None   
grafana/grafana                     v7.2.1          7.2.1      v   None   
vuetifyjs/vuetify                   v0.8.3          0.8.3      v   None   
alibaba/fastjson                    1.2.37         1.2.37   None   None   
psf/requests                        v2.16.3        2.16.3      v   None   
huggingface/transformers            v0.2.0          0.2.0      v   None   
XX-net/XX-Net                       1.14.5         1.14.5   None   None   
                                    1.15.0         1.15.0   None   None   
laravel/laravel                     v4.0.8          4.0.8      v   None   
briannesbitt/Carbon                 1.26.1         1.26.1   None   None   
                                    1.38.3         1.38.3   None   None   
Wox-launcher/Wox                    v1.0.0.185  1.0.0.185      v   None   
dotnet/efcore                       v2.1.8          2.1.8      v   None   
                                    v2.1.13        2.1.13      v   None   
                                    v2.1.16        2.1.16      v   None   
                                    v2.1.17        2.1.17      v   None   
                                    v2.1.18        2.1.18      v   None   
                                    v2.1.20        2.1.20      v   None   
                                    v2.1.21        2.1.21      v   None   
                                    v2.2.7          2.2.7      v   None   
                                    v2.2.8          2.2.8      v   None   
aspnetboilerplate/aspnetboilerplate v0.7.3.0      0.7.3.0      v   None   
                                    v5.10.1        5.10.1      v   None   
                                    v5.12            5.12      v   None   
jellyfin/jellyfin                   v10.0.1        10.0.1      v   None   
SignalR/SignalR                     v0.3.5          0.3.5      v   None   
                                    0.5.0           0.5.0   None   None   
radareorg/radare2                   1.0.0           1.0.0   None   None   
v2ray/v2ray-core                    v0.14.2        0.14.2      v   None   
                                    v2.19.2        2.19.2      v   None   
                                    v2.19.6        2.19.6      v   None   
                                    v2.36.3        2.36.3      v   None   
                                    v2.40.2        2.40.2      v   None   
                                    v2.41            2.41      v   None   
                                    v3.11.3        3.11.3      v   None   
                                    v3.18            3.18      v   None   
                                    v3.22            3.22      v   None   
                                    v3.25            3.25      v   None   
                                    v3.38            3.38      v   None   
                                    v3.46.4        3.46.4      v   None   
istio/istio                         1.0.7           1.0.7   None   None   
                                    1.1.2           1.1.2   None   None   
rclone/rclone                       v1.46.0        1.46.0      v   None   

                                                      lang  \
project                             name                     
microsoft/TypeScript                v1.5.4      typescript   
grafana/grafana                     v7.2.1      typescript   
vuetifyjs/vuetify                   v0.8.3      typescript   
alibaba/fastjson                    1.2.37            java   
psf/requests                        v2.16.3         python   
huggingface/transformers            v0.2.0          python   
XX-net/XX-Net                       1.14.5          python   
                                    1.15.0          python   
laravel/laravel                     v4.0.8            